In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
nltk.download('punkt')
nltk.download('maxent_treebank_pos_tagger')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Read data

In [ ]:
data = pd.read_csv('../input/steamaspect/data-aspect.csv')

## Parse Data

In [ ]:
aspect_data = pd.DataFrame()
arr_word_before = []
arr_word_now = []
arr_word_after = []
arr_pos_tag = []
arr_class = []

for i in range(500):
    text = nltk.word_tokenize(data['review'][i])
    pos_tagged_data = nltk.pos_tag(text)
    aspect = data['aspect_keywords'][i]
    
    for j in range(len(pos_tagged_data)):
        if (j == 0):
            arr_word_before.append('[START]')
        else:
            arr_word_before.append(pos_tagged_data[j-1][0])
        
        arr_word_now.append(pos_tagged_data[j][0])
        
        arr_pos_tag.append(pos_tagged_data[j][1])
        
        if (j == (len(pos_tagged_data) - 1)):
            arr_word_after.append('[END]')
        else:
            arr_word_after.append(pos_tagged_data[j+1][0])
        
        if (pos_tagged_data[j][0] in aspect):
            arr_class.append('true')
        else:
            arr_class.append('false')

aspect_data['word_before'] = arr_word_before
aspect_data['word_now'] = arr_word_now
aspect_data['word_after'] = arr_word_after
aspect_data['pos_tag'] = arr_pos_tag
aspect_data['class'] = arr_class

In [ ]:
 aspect_data.head()

In [ ]:
aspect_data.to_csv('steam-aspect.csv', index=False)